In [17]:
!pip install -U langchain_community langchain-groq langchainhub  langchain  langchain_huggingface

In [18]:
from langchain_community.document_loaders import WebBaseLoader

docs = WebBaseLoader("https://langchain-ai.github.io/langgraph/").load()


In [19]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1200, chunk_overlap=100, add_start_index=True
)
all_splits = text_splitter.split_documents(docs)


In [20]:
from langchain_huggingface import HuggingFaceEmbeddings
!pip install faiss-cpu
from langchain.vectorstores import FAISS
embeddings=HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
# Extract the page_content from the Document objects and create a list of strings
texts = [doc.page_content for doc in all_splits]
vectorstore = FAISS.from_texts(texts=texts, embedding=embeddings) # Pass the list of strings to FAISS.from_texts


In [21]:
question = "what is langgraph?"
retriever = vectorstore.as_retriever(search_type="similarity_score_threshold", search_kwargs={"k": 5, "score_threshold": 0.45})
retrieved_docs = retriever.invoke(question)
context = ' '.join([doc.page_content for doc in retrieved_docs])


In [22]:

from google.colab import userdata
# LLM with function call
from langchain_groq import ChatGroq
import os
groq_api_key=userdata.get('GROQ_API_KEY')
os.environ["GROQ_API_KEY"]=groq_api_key
llm=ChatGroq(groq_api_key=groq_api_key,model_name="Gemma2-9b-It")
response = llm.invoke(f"""Answer the question according to the context given very briefly:
           Question: {question}.
           Context: {context}
""")
response

AIMessage(content='LangGraph is a platform for deploying and managing "agentic applications", built on the open-source LangGraph framework.  It provides tools for development, deployment, debugging, and monitoring. \n', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 42, 'prompt_tokens': 407, 'total_tokens': 449, 'completion_time': 0.076363636, 'prompt_time': 0.016066569, 'queue_time': 0.0040659089999999995, 'total_time': 0.092430205}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-8d322953-19a9-40f8-93e3-6edb6184744b-0', usage_metadata={'input_tokens': 407, 'output_tokens': 42, 'total_tokens': 449})

In [23]:
response.content

'LangGraph is a platform for deploying and managing "agentic applications", built on the open-source LangGraph framework.  It provides tools for development, deployment, debugging, and monitoring. \n'